In [4]:
# Import, preparation/processing and dataset - cats v dogs
# Data loader, transformation
# NN, activation -> class, __init__
# Loss and optimizer
# Training loop
# Model evaluate

# Processing and dataset - cats v dogs
import os
import cv2
import numpy as np
from tqdm import tqdm # for observing the progress

In [5]:
rebuild_data = True

class catsvdogs():
    size = 50
    cats = r"C:\Users\Gokul\Downloads\PetImages\Cat"
    dogs = r"C:\Users\Gokul\Downloads\PetImages\Dog"
    process_imgs = r"C:\Users\Gokul\Downloads\PetImages\Data"
    classes = {cats: 0, dogs: 1}
    data = []
    catcount = 0
    dogcount = 0

    def make_data(self):
        for label in self.classes:                       # iterate over directories
            for f in tqdm(os.listdir(label)):           # iterate through imgs in directory
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)   # join label and image
                        img = cv2.imread(path, 0)
                        img = cv2.resize(img, (self.size, self.size))
                        self.data.append([np.array(img), np.eye(2)[self.classes[label]]]) # eye for one-hot encode

                        if label == self.cats:
                            self.catcount += 1
                        elif label == self.dogs:
                            self.dogcount += 1
                    
                    except Exception as e:
                        pass
                    
        np.random.shuffle(self.data)
        np.save("Rebulit_data.npy", self.data)
        print("Cats_count: ", catsVdogs.catcount)
        print("Dogs_count: ", catsVdogs.dogcount)


In [6]:
if rebuild_data:
    catsVdogs = catsvdogs()
    catsVdogs.make_data()

data_processed = np.load("Rebulit_data.npy", allow_pickle = True) 
print(len(data_processed))

100%|███████████████████████████████████████████████████████████████████████████| 12501/12501 [00:34<00:00, 359.82it/s]


Cats_count:  12476
Dogs_count:  12470
24946


In [7]:
import torch 
import torch.nn as nn
import torch.nn.functional as func
import torchvision
import torchvision.transforms as transforms

# Device configuration
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 50
batch_size = 128
no_classes = 2
no_epochs = 1
learning_rate = 0.001

In [8]:
# Data 
x_input_image = torch.Tensor([i[0] for i in data_processed]).view(-1, input_size, input_size)
x_input_image = x_input_image/255.0     # scaling the pixel values
y_output_label = torch.Tensor([i[1] for i in data_processed])

In [9]:
# Data split/loader

split_percent = 0.1
split_size = int(len(data_processed)*split_percent)
train_x = x_input_image[:-split_size]
train_y = y_output_label[:-split_size]

test_x = x_input_image[-split_size:]
test_y = y_output_label[-split_size:]

print(len(train_x), len(test_x))

22452 2494


In [10]:
# NN
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)       # 1x50x50  -> 32x46x46(conv1) -> 32x23x23(pool)
        self.conv2 = nn.Conv2d(32, 64, 5)      # 32x23x23 -> 64x19x19(conv2) -> 64x9x9  (pool)
        self.conv3 = nn.Conv2d(64, 128, 5)     # 64x9x9   -> 128x5x5 (conv3) -> 128x2x2 (pool)
        self.pool = nn.MaxPool2d(2, 2)     
        self.fc1 = nn.Linear(128*2*2, 512)
        self.fc2 = nn.Linear(512, no_classes)

    def forward(self, x):
        x = self.pool(func.relu(self.conv1(x)))   
        x = self.pool(func.relu(self.conv2(x)))  
        x = self.pool(func.relu(self.conv3(x)))
        x = x.view(-1, 128*2*2)
        x = func.relu(self.fc1(x))
        x = self.fc2(x)
        return func.softmax(x, dim = 1)

model = ConvNet()
print(model)

# Loss function and optimizer
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

ConvNet(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [11]:
# Training loop
for epoch in range(no_epochs):
    for data in tqdm(range(0, len(train_x), batch_size)):
        # Resizing
        x = train_x[data: data + batch_size].view(-1, 1, input_size, input_size)
        y = train_y[data: data + batch_size]

        # Forward
        output = model(x)
        loss = loss_func(output, y)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


100%|████████████████████████████████████████████████████████████████████████████████| 176/176 [00:52<00:00,  3.37it/s]


In [12]:
# Test/Model evaluate
with torch.no_grad():
    correct = 0
    total = 0
    for i in tqdm(range(len(test_x))):
        real_class = torch.argmax(test_y[i])
        net_out = model(test_x[i].view(-1, 1, input_size, input_size))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

100%|█████████████████████████████████████████████████████████████████████████████| 2494/2494 [00:05<00:00, 487.95it/s]

Accuracy:  0.648
